In [359]:
from keras_preprocessing.image import ImageDataGenerator
# Initialize Neural network
from keras.models import Sequential
# Create layers in ANN
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

import pandas as pd
import numpy as np

In [326]:
df = pd.read_csv('C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')

In [451]:
from sklearn.utils import shuffle

df_train = df[df.Dataset_type == 'TRAIN']
df_train = df_train[(df.Label_2_Virus_category == 'COVID-19') & (df.Label == 'Pnemonia') | (df.Label == 'Normal')]
df_train = shuffle(df_train, random_state=1)

df_val = df_train.iloc[1000:]
df_train = df_train.iloc[:1000]

df_test = df[df.Dataset_type == 'TEST']


C:\Users\ksa\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [275]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(1000, 7)
(400, 7)
(624, 7)


In [284]:
df_train.Label.value_counts()
df_val.Label.value_counts()

Normal      377
Pnemonia     23
Name: Label, dtype: int64

In [303]:
datagen=ImageDataGenerator(rescale=1./255.)
# datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/",
x_col="X_ray_image_name",
y_col="Label",
# subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(32,32))


valid_generator=datagen.flow_from_dataframe(
dataframe=df_val,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/",
x_col="X_ray_image_name",
y_col="Label",
# subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(32,32))


test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test",
x_col="X_ray_image_name",
y_col="Label",
batch_size=32,
seed=42,
shuffle=False,
class_mode="binary",
target_size=(32,32))


Found 1000 validated image filenames belonging to 2 classes.
Found 400 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [295]:
from keras.metrics import *

METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
]

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=METRICS)
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_56 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_57 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 15, 15, 64)      

In [324]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


## CNN Model 

In [297]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=15
)

Epoch 1/15
31/31 [==============================] - 67s 2s/step - loss: 0.1916 - accuracy: 0.9432 - precision: 0.0417 - recall: 0.0303 - auc: 0.5316 - val_loss: 0.1509 - val_accuracy: 0.9427 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7671
Epoch 2/15
31/31 [==============================] - 89s 3s/step - loss: 0.1468 - accuracy: 0.9638 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6713 - val_loss: 0.2354 - val_accuracy: 0.9457 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.8770
Epoch 3/15
31/31 [==============================] - 85s 3s/step - loss: 0.1296 - accuracy: 0.9680 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7308 - val_loss: 0.2658 - val_accuracy: 0.9429 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.9804
Epoch 4/15
31/31 [==============================] - 81s 3s/step - loss: 0.1398 - accuracy: 0.9628 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7555 - val_loss: 0.3053 - val_accuracy: 0.9429

## Image Augmentation

Deep networks need large amount of training data to achieve good performance. To build a powerful image classifier using very little training data, image augmentation is usually required to boost the performance of deep networks. Image augmentation artificially creates training images through different ways of processing or combination of multiple processing, such as random rotation, shifts, shear and flips, etc

We will be using keras ImageDataGenerator's inbuilt image augumentation functionality for the process of image augumentation

In [314]:
# Image Augmentation for training dataset
datagen=ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=[0.9, 1.25],
    brightness_range=[0.5, 1.5]
)

test_datagen=ImageDataGenerator(rescale=1./255.)

# datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/",
x_col="X_ray_image_name",
y_col="Label",
# subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(32,32))


valid_generator=datagen.flow_from_dataframe(
dataframe=df_val,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/",
x_col="X_ray_image_name",
y_col="Label",
# subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(32,32))


test_generator=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test",
x_col="X_ray_image_name",
y_col="Label",
batch_size=32,
seed=42,
shuffle=False,
class_mode="binary",
target_size=(32,32))


Found 1000 validated image filenames belonging to 2 classes.
Found 400 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [315]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=15
)

Epoch 1/15


C:\Users\ksa\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\ksa\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


31/31 [==============================] - 32s 1s/step - loss: 0.1757 - accuracy: 0.9463 - precision: 0.0500 - recall: 0.0294 - auc: 0.7262 - val_loss: 0.1670 - val_accuracy: 0.9453 - val_precision: 0.7500 - val_recall: 0.1304 - val_auc: 0.8807
Epoch 2/15
31/31 [==============================] - 41s 1s/step - loss: 0.1433 - accuracy: 0.9659 - precision: 0.7500 - recall: 0.0857 - auc: 0.7448 - val_loss: 0.1810 - val_accuracy: 0.9511 - val_precision: 0.7500 - val_recall: 0.1500 - val_auc: 0.8899
Epoch 3/15
31/31 [==============================] - 42s 1s/step - loss: 0.1352 - accuracy: 0.9649 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7595 - val_loss: 0.3062 - val_accuracy: 0.9457 - val_precision: 1.0000 - val_recall: 0.0909 - val_auc: 0.9461
Epoch 4/15
31/31 [==============================] - 40s 1s/step - loss: 0.1271 - accuracy: 0.9680 - precision: 1.0000 - recall: 0.0312 - auc: 0.7867 - val_loss: 0.4889 - val_accuracy: 0.9511 - val_precision: 1.0000 - val_recall: 0.0526 - val

## Correcting the imbalance Data
- We will remove 4 / 5 th of the Normal labelled images while keeping the count of COVID-19 labelled images same
- 1342 / 5 ~ 269; 269 / (269 + 58) ~ 82.26 %
- To compensate for less number of training images, we increase the number of epochs

In [327]:
df_train = df[df.Dataset_type == 'TRAIN']
df_train = df_train[(df.Label_2_Virus_category == 'COVID-19') & (df.Label == 'Pnemonia') | (df.Label == 'Normal')]
df_train = shuffle(df_train, random_state=1)

# Undersampling the 0's (normal data)
df_train_normal = df_train[df_train['Label']=='Normal']
df_train_Covid = df_train[df_train['Label']=='Pnemonia']

df_train_frac_normal = df_train_normal.sample(frac=(1/5))

# Using pd.concat to concat undersampled Normal data and Covid dataset
df_train_concat = pd.concat([df_train_frac_normal,df_train_Covid ], axis = 0)
df_train_concat = shuffle(df_train_concat, random_state = 0)


df_balanced_val = df_train_concat.iloc[240:]
df_balanced_train = df_train_concat.iloc[:240]

df_test = df[df.Dataset_type == 'TEST']


C:\Users\ksa\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [328]:
train_generator=datagen.flow_from_dataframe(
dataframe=df_balanced_train,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/",
x_col="X_ray_image_name",
y_col="Label",
# subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(32,32))


valid_generator=datagen.flow_from_dataframe(
dataframe=df_balanced_val,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/",
x_col="X_ray_image_name",
y_col="Label",
# subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(32,32))



Found 240 validated image filenames belonging to 2 classes.
Found 86 validated image filenames belonging to 2 classes.


In [331]:
from keras.metrics import *

METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
]

balanced_model = Sequential()
balanced_model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
balanced_model.add(Activation('relu'))
balanced_model.add(Conv2D(32, (3, 3)))
balanced_model.add(Activation('relu'))
balanced_model.add(MaxPooling2D(pool_size=(2, 2)))
balanced_model.add(Dropout(0.25))
balanced_model.add(Conv2D(64, (3, 3), padding='same'))
balanced_model.add(Activation('relu'))
balanced_model.add(Conv2D(64, (3, 3)))
balanced_model.add(Activation('relu'))
balanced_model.add(MaxPooling2D(pool_size=(2, 2)))
balanced_model.add(Dropout(0.25))
balanced_model.add(Flatten())
balanced_model.add(Dense(512))
balanced_model.add(Activation('relu'))
balanced_model.add(Dropout(0.5))
balanced_model.add(Dense(1, activation='sigmoid'))
balanced_model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=METRICS)
balanced_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_57 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_66 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_67 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 15, 15, 64)      

In [334]:
balanced_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=30
)

Epoch 1/30
31/31 [==============================] - 82s 3s/step - loss: 0.4695 - accuracy: 0.8448 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4308 - val_loss: 0.6297 - val_accuracy: 0.7558 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4883
Epoch 2/30
31/31 [==============================] - 79s 3s/step - loss: 0.4577 - accuracy: 0.8470 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4953 - val_loss: 0.5912 - val_accuracy: 0.7558 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5895
Epoch 3/30
31/31 [==============================] - 76s 2s/step - loss: 0.4372 - accuracy: 0.8481 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5241 - val_loss: 0.4987 - val_accuracy: 0.7558 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5657
Epoch 4/30
31/31 [==============================] - 73s 2s/step - loss: 0.4380 - accuracy: 0.8427 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5822 - val_loss: 0.5335 - val_accuracy

## Prediction on Test Set

In [ ]:
# Predict the model on test set
test_generator.reset()
pred=balanced_model.predict_generator(test_generator,
steps=STEP_SIZE_TEST+1,
verbose=1)

# change the array to 0,1
predicted_class_indices=np.argmax(pred,axis=1)

# Extract filename and classes from test data
filenames=test_generator.filenames
classes = test_generator.classes

# create actual and predicted data frame
Actuals = pd.DataFrame({"Filename":filenames,
                     "Class":classes})

Results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predicted_class_indices}
                     )
# convert actual and predict df to list for evaluation
Act_Cat = Actuals['Class'].values.tolist()
Pred_Cat = Results['Predictions'].values.tolist()

In [354]:
Actuals.Class.value_counts()
Results.Predictions.value_counts()

0    624
Name: Predictions, dtype: int64

In [355]:
Act_Cat = Actuals['Class'].values.tolist()
Pred_Cat = Results['Predictions'].values.tolist()

In [356]:
# Evaluate the model to get the corresponding Accuracy, Precision, and Recall
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

CM =confusion_matrix(Act_Cat,Pred_Cat)
# print("Confusion Matrix :")
print(CM)



[[234   0]
 [390   0]]


In [357]:
# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# # predictions = [labels[k] for k in predicted_class_indices]

# TRANSFER LEARNING
### Build a Fine-Tuned VGG16 model

In [430]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense

import numpy as np

In [445]:
img_width, img_height = 64, 64
vgg16 = VGG16(include_top=False, weights=None, input_shape=(img_width, img_height, 3))

In [446]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [448]:
# use transfer learning for re-training the last layers
# Freeze first 25 layers, so that we can retrain 26th and so on using our classes.
for layer in vgg16.layers[:-2]:
   layer.trainable = False

#Add a layer where input is the output of the  second last layer 
Z = Flatten(name='flatten')(vgg16.layers[-1].output)
New = Dense(1, activation='sigmoid', name='predictions')(Z)


#Then create the corresponding model 
my_model = Model(input=vgg16.input, output=New)
my_model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0  

C:\Users\ksa\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  if sys.path[0] == '':


In [455]:
from keras.metrics import *

METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
]
# my_model.compile(loss = "binary_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=METRICS)
my_model.compile(loss = "binary_crossentropy", optimizer = "ADAM", metrics=METRICS)

In [459]:
img_width = 64
img_height = 64

train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/",
x_col="X_ray_image_name",
y_col="Label",
# subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(img_width,img_height))


valid_generator=datagen.flow_from_dataframe(
dataframe=df_val,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/",
x_col="X_ray_image_name",
y_col="Label",
# subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="binary",
target_size=(img_width,img_height))

test_generator=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory="C:/Users/ksa/OneDrive - Delta Partners/Documents/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test",
x_col="X_ray_image_name",
y_col="Label",
batch_size=32,
seed=42,
shuffle=False,
class_mode="binary",
target_size=(img_width,img_height))



Found 1000 validated image filenames belonging to 2 classes.
Found 400 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [457]:
my_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=30
)

Epoch 1/30
31/31 [==============================] - 62s 2s/step - loss: 0.2762 - accuracy: 0.9556 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4845 - val_loss: 0.1671 - val_accuracy: 0.9401 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4295
Epoch 2/30
31/31 [==============================] - 73s 2s/step - loss: 0.2092 - accuracy: 0.9638 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4054 - val_loss: 0.2373 - val_accuracy: 0.9429 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5477
Epoch 3/30
31/31 [==============================] - 76s 2s/step - loss: 0.1547 - accuracy: 0.9669 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5072 - val_loss: 0.2617 - val_accuracy: 0.9429 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5304
Epoch 4/30
31/31 [==============================] - 65s 2s/step - loss: 0.1858 - accuracy: 0.9607 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.3735 - val_loss: 0.0438 - val_accuracy

In [460]:
# Predict the model on test set
test_generator.reset()
pred=my_model.predict_generator(test_generator,
steps=STEP_SIZE_TEST+1,
verbose=1)

# change the array to 0,1
predicted_class_indices=np.argmax(pred,axis=1)

# Extract filename and classes from test data
filenames=test_generator.filenames
classes = test_generator.classes

# create actual and predicted data frame
Actuals = pd.DataFrame({"Filename":filenames,
                     "Class":classes})

Results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predicted_class_indices}
                     )
# convert actual and predict df to list for evaluation
Act_Cat = Actuals['Class'].values.tolist()
Pred_Cat = Results['Predictions'].values.tolist()

20/20 [==============================] - 18s 914ms/step


In [461]:
# Evaluate the model to get the corresponding Accuracy, Precision, and Recall
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

CM =confusion_matrix(Act_Cat,Pred_Cat)
# print("Confusion Matrix :")
print(CM)



[[234   0]
 [390   0]]
